### RMSLE : 49.31338725928%
### Train Accuracy : 98.09 %
### Test Accuracy : 84.97 %
### R^2 : 84.97420992292601%

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("train.csv")
meal_info = pd.read_csv("meal_info.csv")
fulfilment_center_info = pd.read_csv("fulfilment_center_info.csv")

In [3]:
df = df.merge(meal_info,on='meal_id', how = 'left')
df = df.merge(fulfilment_center_info, on='center_id', how = 'left')

In [4]:
df.shape

(456548, 15)

In [5]:
df_new = df.copy() # using a deep copy technique for making a copy of the data

In [6]:
df_new['discount'] = df_new['base_price'] - df_new['checkout_price']

In [7]:
df_new['discount_percent'] = ((df_new['base_price'] - df_new['checkout_price'])/df_new['base_price'])*100

In [8]:
df_new['Quarter']=(df_new['week']/13).astype('int64')

In [9]:
df_new['Quarter']=df_new['Quarter'].map({0:'Q1',1:'Q2',2:'Q3',3:'Q4',4:'Q1',5:'Q2',6:'Q3',7:'Q4',8:'Q1',9:'Q2',10:'Q3',11:'Q4'})

In [10]:
df_new['Year']=(df_new['week']/52).astype('int64')

In [11]:
df_new['Year'] = df_new['Year'].map({0:'Y1', 1:'Y2', 2:'Y3'})

In [12]:
df_new

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,category,cuisine,city_code,region_code,center_type,op_area,discount,discount_percent,Quarter,Year
0,1379560,1,55,1885,136.83,152.29,0,0,177,Beverages,Thai,647,56,TYPE_C,2.0,15.46,10.151684,Q1,Y1
1,1466964,1,55,1993,136.83,135.83,0,0,270,Beverages,Thai,647,56,TYPE_C,2.0,-1.00,-0.736214,Q1,Y1
2,1346989,1,55,2539,134.86,135.86,0,0,189,Beverages,Thai,647,56,TYPE_C,2.0,1.00,0.736052,Q1,Y1
3,1338232,1,55,2139,339.50,437.53,0,0,54,Beverages,Indian,647,56,TYPE_C,2.0,98.03,22.405321,Q1,Y1
4,1448490,1,55,2631,243.50,242.50,0,0,40,Beverages,Indian,647,56,TYPE_C,2.0,-1.00,-0.412371,Q1,Y1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456543,1271326,145,61,1543,484.09,484.09,0,0,68,Desert,Indian,473,77,TYPE_A,4.5,0.00,0.000000,Q4,Y3
456544,1062036,145,61,2304,482.09,482.09,0,0,42,Desert,Indian,473,77,TYPE_A,4.5,0.00,0.000000,Q4,Y3
456545,1110849,145,61,2664,237.68,321.07,0,0,501,Salad,Italian,473,77,TYPE_A,4.5,83.39,25.972529,Q4,Y3
456546,1147725,145,61,2569,243.50,313.34,0,0,729,Salad,Italian,473,77,TYPE_A,4.5,69.84,22.288887,Q4,Y3


In [13]:
data = pd.get_dummies(df_new, columns=["category", "cuisine","center_type",'Quarter', 'Year'], prefix=["category", "cuisine","center_type",'Quarter', 'Year'])

In [14]:
X = data.drop("num_orders", axis=1)  # X variable has the all the features except num_orders which is target column
y = data['num_orders'] # num_order is the target column so, y is the target variable

In [15]:
X = X.drop(['id'], axis =1)

In [16]:
X

,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,city_code,region_code,op_area,...,center_type_TYPE_A,center_type_TYPE_B,center_type_TYPE_C,Quarter_Q1,Quarter_Q2,Quarter_Q3,Quarter_Q4,Year_Y1,Year_Y2,Year_Y3
0,1,55,1885,136.83,152.29,0,0,647,56,2.0,...,0,0,1,1,0,0,0,1,0,0
1,1,55,1993,136.83,135.83,0,0,647,56,2.0,...,0,0,1,1,0,0,0,1,0,0
2,1,55,2539,134.86,135.86,0,0,647,56,2.0,...,0,0,1,1,0,0,0,1,0,0
3,1,55,2139,339.50,437.53,0,0,647,56,2.0,...,0,0,1,1,0,0,0,1,0,0
4,1,55,2631,243.50,242.50,0,0,647,56,2.0,...,0,0,1,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456543,145,61,1543,484.09,484.09,0,0,473,77,4.5,...,1,0,0,0,0,0,1,0,0,1
456544,145,61,2304,482.09,482.09,0,0,473,77,4.5,...,1,0,0,0,0,0,1,0,0,1
456545,145,61,2664,237.68,321.07,0,0,473,77,4.5,...,1,0,0,0,0,0,1,0,0,1
456546,145,61,2569,243.50,313.34,0,0,473,77,4.5,...,1,0,0,0,0,0,1,0,0,1


In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [19]:
print("X_trian",X_train.shape)
print("X_test",X_test.shape)
print("y_train",y_train.shape)
print("y_test",y_test.shape)

X_trian (342411, 40)
X_test (114137, 40)
y_train (342411,)
y_test (114137,)


# --------------------------------------------------------------------------------------------------------------

In [20]:
from sklearn.ensemble import RandomForestRegressor

In [21]:
reg4 = RandomForestRegressor(n_estimators=500, random_state = 42)

In [22]:
reg4.fit(X_train, y_train)

RandomForestRegressor(n_estimators=500, random_state=42)

In [23]:
y_pred4 = reg4.predict(X_test)

In [24]:
accuracy4 = round(reg4.score(X_train, y_train)*100,2)
print("Train Accuracy :",round(accuracy4, 2), '%')

Train Accuracy : 98.09 %


In [25]:
accuracy4 = round(reg4.score(X_test, y_test)*100,2)
print("Test Accuracy :",round(accuracy4, 2), '%')

Test Accuracy : 84.97 %


In [27]:
from sklearn.metrics import mean_squared_log_error

print('RMSLE :',np.sqrt(mean_squared_log_error(y_test,y_pred4)))

RMSLE : 0.4931338725928


In [28]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred4)

0.8497420992292601

# --------------------------------------------------------------------------------------------------------------

In [22]:
from sklearn.ensemble import RandomForestRegressor

In [21]:
from sklearn.model_selection import RandomizedSearchCV

In [23]:
n_estimators = [int(x) for x in np.linspace(start = 300, stop = 600, num = 10)] # Number of trees in random forest
max_features = ['auto', 'sqrt'] # Number of features to consider at every split
max_depth = [int(x) for x in np.linspace(10, 55, num = 11)] # Maximum number of levels in tree
max_depth.append(None)
min_samples_split = [2, 5, 10] # Minimum number of samples required to split a node
min_samples_leaf = [1, 2, 4] # Minimum number of samples required at each leaf node
bootstrap = [True, False] # Method of selecting samples for training each tree
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [300, 333, 366, 400, 433, 466, 500, 533, 566, 600], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 14, 19, 23, 28, 32, 37, 41, 46, 50, 55, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [24]:
regressor = RandomForestRegressor() # Random search of parameters, using 3 fold cross validation, 
rf_random = RandomizedSearchCV(estimator = regressor, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [ ]:
rf_random.fit(X_train, y_train) # Fit the random search model

Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [ ]:
rf_random.best_params_

In [ ]:
reg5 = RandomForestRegressor(n_estimators = 311, min_samples_split = 2, min_samples_leaf = 4, max_features = 'auto',
                                      max_depth = 23, bootstrap = True)

In [ ]:
reg5.fit(X_train, y_train)

In [ ]:
y_pred5 = reg5.predict(X_test)

In [ ]:
accuracy5 = round(reg5.score(X_train, y_train)*100,2)
print(round(accuracy5, 2), '%'5

In [ ]:
accuracy5 = round(reg5.score(X_test, y_test)*100,2)
print(round(accuracy5, 2), '%')

In [ ]:
print('RMSLE :',np.sqrt(mean_squared_error(y_test,y_pred5)))

### ----------------------------------------------------------------- Testing Data----------------------------------------------------------------------------

In [69]:
test_data = pd.read_csv("test.csv")

In [70]:
test_data = test_data.merge(meal_info,on='meal_id', how = 'left')
test_data = test_data.merge(fulfilment_center_info, on='center_id', how = 'left')

In [71]:
test_data.shape

(32573, 14)

In [72]:
test_data['discount'] = test_data['base_price'] - test_data['checkout_price']

In [73]:
test_data['discount_percent'] = ((test_data['base_price'] - test_data['checkout_price'])/test_data['base_price'])*100

In [74]:
test_data['Quarter']=(test_data['week']/13).astype('int64')

In [75]:
test_data['Quarter']=test_data['Quarter'].map({0:'Q1',1:'Q2',2:'Q3',3:'Q4',4:'Q1',5:'Q2',6:'Q3',7:'Q4',8:'Q1',9:'Q2',10:'Q3',11:'Q4'})

In [76]:
test_data['Year']=(test_data['week']/52).astype('int64')

In [77]:
test_data['Year'] = test_data['Year'].map({0:'Y1', 1:'Y2', 2:'Y3'})

In [78]:
test_data

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,category,cuisine,city_code,region_code,center_type,op_area,discount,discount_percent,Quarter,Year
0,1028232,146,55,1885,158.11,159.11,0,0,Beverages,Thai,647,56,TYPE_C,2.0,1.0,0.628496,Q4,Y3
1,1127204,146,55,1993,160.11,159.11,0,0,Beverages,Thai,647,56,TYPE_C,2.0,-1.0,-0.628496,Q4,Y3
2,1212707,146,55,2539,157.14,159.14,0,0,Beverages,Thai,647,56,TYPE_C,2.0,2.0,1.256755,Q4,Y3
3,1082698,146,55,2631,162.02,162.02,0,0,Beverages,Indian,647,56,TYPE_C,2.0,0.0,0.000000,Q4,Y3
4,1400926,146,55,1248,163.93,163.93,0,0,Beverages,Indian,647,56,TYPE_C,2.0,0.0,0.000000,Q4,Y3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32568,1250239,155,61,1543,482.09,484.09,0,0,Desert,Indian,473,77,TYPE_A,4.5,2.0,0.413146,Q4,Y3
32569,1039516,155,61,2304,483.09,483.09,0,0,Desert,Indian,473,77,TYPE_A,4.5,0.0,0.000000,Q4,Y3
32570,1158107,155,61,2664,322.07,323.07,0,0,Salad,Italian,473,77,TYPE_A,4.5,1.0,0.309530,Q4,Y3
32571,1444235,155,61,2569,322.07,323.07,0,0,Salad,Italian,473,77,TYPE_A,4.5,1.0,0.309530,Q4,Y3


In [79]:
data1 = pd.get_dummies(test_data, columns=["category", "cuisine","center_type",'Quarter', 'Year'], prefix=["category", "cuisine","center_type",'Quarter', 'Year'])

In [80]:
data1

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,city_code,region_code,...,category_Starters,cuisine_Continental,cuisine_Indian,cuisine_Italian,cuisine_Thai,center_type_TYPE_A,center_type_TYPE_B,center_type_TYPE_C,Quarter_Q4,Year_Y3
0,1028232,146,55,1885,158.11,159.11,0,0,647,56,...,0,0,0,0,1,0,0,1,1,1
1,1127204,146,55,1993,160.11,159.11,0,0,647,56,...,0,0,0,0,1,0,0,1,1,1
2,1212707,146,55,2539,157.14,159.14,0,0,647,56,...,0,0,0,0,1,0,0,1,1,1
3,1082698,146,55,2631,162.02,162.02,0,0,647,56,...,0,0,1,0,0,0,0,1,1,1
4,1400926,146,55,1248,163.93,163.93,0,0,647,56,...,0,0,1,0,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32568,1250239,155,61,1543,482.09,484.09,0,0,473,77,...,0,0,1,0,0,1,0,0,1,1
32569,1039516,155,61,2304,483.09,483.09,0,0,473,77,...,0,0,1,0,0,1,0,0,1,1
32570,1158107,155,61,2664,322.07,323.07,0,0,473,77,...,0,0,0,1,0,1,0,0,1,1
32571,1444235,155,61,2569,322.07,323.07,0,0,473,77,...,0,0,0,1,0,1,0,0,1,1


In [81]:
test_data = data1.drop(['id'], axis =1)

In [82]:
test_data

,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,city_code,region_code,op_area,...,category_Starters,cuisine_Continental,cuisine_Indian,cuisine_Italian,cuisine_Thai,center_type_TYPE_A,center_type_TYPE_B,center_type_TYPE_C,Quarter_Q4,Year_Y3
0,146,55,1885,158.11,159.11,0,0,647,56,2.0,...,0,0,0,0,1,0,0,1,1,1
1,146,55,1993,160.11,159.11,0,0,647,56,2.0,...,0,0,0,0,1,0,0,1,1,1
2,146,55,2539,157.14,159.14,0,0,647,56,2.0,...,0,0,0,0,1,0,0,1,1,1
3,146,55,2631,162.02,162.02,0,0,647,56,2.0,...,0,0,1,0,0,0,0,1,1,1
4,146,55,1248,163.93,163.93,0,0,647,56,2.0,...,0,0,1,0,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32568,155,61,1543,482.09,484.09,0,0,473,77,4.5,...,0,0,1,0,0,1,0,0,1,1
32569,155,61,2304,483.09,483.09,0,0,473,77,4.5,...,0,0,1,0,0,1,0,0,1,1
32570,155,61,2664,322.07,323.07,0,0,473,77,4.5,...,0,0,0,1,0,1,0,0,1,1
32571,155,61,2569,322.07,323.07,0,0,473,77,4.5,...,0,0,0,1,0,1,0,0,1,1


In [83]:
test_data.columns

Index(['week', 'center_id', 'meal_id', 'checkout_price', 'base_price',
       'emailer_for_promotion', 'homepage_featured', 'city_code',
       'region_code', 'op_area', 'discount', 'discount_percent',
       'category_Beverages', 'category_Biryani', 'category_Desert',
       'category_Extras', 'category_Fish', 'category_Other Snacks',
       'category_Pasta', 'category_Pizza', 'category_Rice Bowl',
       'category_Salad', 'category_Sandwich', 'category_Seafood',
       'category_Soup', 'category_Starters', 'cuisine_Continental',
       'cuisine_Indian', 'cuisine_Italian', 'cuisine_Thai',
       'center_type_TYPE_A', 'center_type_TYPE_B', 'center_type_TYPE_C',
       'Quarter_Q4', 'Year_Y3'],
      dtype='object')

In [84]:
X.columns

Index(['week', 'center_id', 'meal_id', 'checkout_price', 'base_price',
       'emailer_for_promotion', 'homepage_featured', 'city_code',
       'region_code', 'op_area', 'discount', 'discount_percent',
       'category_Beverages', 'category_Biryani', 'category_Desert',
       'category_Extras', 'category_Fish', 'category_Other Snacks',
       'category_Pasta', 'category_Pizza', 'category_Rice Bowl',
       'category_Salad', 'category_Sandwich', 'category_Seafood',
       'category_Soup', 'category_Starters', 'cuisine_Continental',
       'cuisine_Indian', 'cuisine_Italian', 'cuisine_Thai',
       'center_type_TYPE_A', 'center_type_TYPE_B', 'center_type_TYPE_C',
       'Quarter_Q1', 'Quarter_Q2', 'Quarter_Q3', 'Quarter_Q4', 'Year_Y1',
       'Year_Y2', 'Year_Y3'],
      dtype='object')

In [85]:
test_data['Quarter_Q1'] = 0 

In [86]:
test_data['Quarter_Q2'] = 0 

In [87]:
test_data['Quarter_Q3'] = 0 

In [88]:
test_data['Year_Y1'] = 0 

In [89]:
test_data['Year_Y2'] = 0 

In [90]:
test_data

,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,city_code,region_code,op_area,...,center_type_TYPE_A,center_type_TYPE_B,center_type_TYPE_C,Quarter_Q4,Year_Y3,Quarter_Q1,Quarter_Q2,Quarter_Q3,Year_Y1,Year_Y2
0,146,55,1885,158.11,159.11,0,0,647,56,2.0,...,0,0,1,1,1,0,0,0,0,0
1,146,55,1993,160.11,159.11,0,0,647,56,2.0,...,0,0,1,1,1,0,0,0,0,0
2,146,55,2539,157.14,159.14,0,0,647,56,2.0,...,0,0,1,1,1,0,0,0,0,0
3,146,55,2631,162.02,162.02,0,0,647,56,2.0,...,0,0,1,1,1,0,0,0,0,0
4,146,55,1248,163.93,163.93,0,0,647,56,2.0,...,0,0,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32568,155,61,1543,482.09,484.09,0,0,473,77,4.5,...,1,0,0,1,1,0,0,0,0,0
32569,155,61,2304,483.09,483.09,0,0,473,77,4.5,...,1,0,0,1,1,0,0,0,0,0
32570,155,61,2664,322.07,323.07,0,0,473,77,4.5,...,1,0,0,1,1,0,0,0,0,0
32571,155,61,2569,322.07,323.07,0,0,473,77,4.5,...,1,0,0,1,1,0,0,0,0,0


In [91]:
y_pred = reg4.predict(test_data)

In [92]:
y_pred

array([ 83.558,  72.044, 129.576, ..., 309.776, 379.932, 162.822])

In [93]:
prediction = pd.DataFrame(y_pred)

In [94]:
prediction

,0
0,83.558
1,72.044
2,129.576
3,75.078
4,65.598
...,...
32568,74.806
32569,31.238
32570,309.776
32571,379.932


In [99]:
dataframe = pd.read_csv("test.csv")

In [100]:
Result=pd.DataFrame(y_pred)
#Result=np.expm1(Result).astype('int64')
Submission = pd.DataFrame(columns=['id', 'num_orders'])
Submission['id'] = dataframe['id']
Submission['num_orders'] = Result.values
Submission.to_csv('Submission.csv', index=False)

### RMSLE : 49.31338725928%
### Train Accuracy : 98.09 %
### Test Accuracy : 84.97 %
### R^2 : 84.97420992292601%